In [6]:
!pip install download

In [8]:
from download import download

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download(url, "./data/", kind="zip", replace=True)


Creating data folder...

file_sizes: 100%|██████████████████████████| 10.8M/10.8M [00:00<00:00, 14.5MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./data/


In [21]:
import os

import mindspore
from mindspore.common.initializer import Normal
from mindspore.dataset import MnistDataset, vision
from mindspore import nn, LossMonitor, TimeMonitor
from mindspore.dataset.vision import Inter
from mindspore.train import Model, CheckpointConfig, ModelCheckpoint, Callback
import mindspore.dataset.transforms as transforms
from mindspore import dtype as mstype
mindspore.set_context(mode=mindspore.GRAPH_MODE)

In [26]:
def create_dataset(data_path, Train = True,batch_size=32, repeat_size=1):
    """
    创建用于训练的MNIST数据集。

    此函数负责加载MNIST数据集，对数据进行预处理和转换，以便它们可以用于训练神经网络。数据预处理包括调整图像大小、重新缩放和类型转换。

    参数:
        data_path (str): MNIST数据集的路径。这应该是包含MNIST数据文件的目录路径。
        Train (bool):加载什么数据集，若为True 加载训练数据集，否则加载测试数据集
        batch_size (int, 可选): 每个数据批次的大小。默认值为32。
        repeat_size (int, 可选): 数据集重复的次数。这用于增加数据集的大小。默认值为1。

    步骤:
        1. 加载MNIST数据集。
        2. 对图像执行大小调整操作，将图像大小统一调整为32x32像素。
        3. 对图像进行重新缩放和标准化处理。先将像素值缩放到0-1之间，然后进行标准化。
        4. 将图像的格式从高宽通道(HWC)转换为通道高宽(CHW)。
        5. 对标签进行类型转换，将其转换为整型（int32）。
        6. 对数据集进行洗牌、批处理和重复操作，以准备训练过程。

    返回:
        返回一个处理过的MNIST数据集，可以直接用于模型训练。

    注意:
        - 数据集的预处理步骤对于训练深度学习模型来说是非常重要的，它们会影响训练的效果和速度。
        - 调整batch_size和repeat_size可以影响模型训练时的内存消耗和速度。
    """
    data_train = os.path.join(data_path, 'train')
    data_test = os.path.join(data_path, 'test')
    mnist_dataset = MnistDataset(data_train if Train==True else data_test )

    resize_operation = vision.Resize((32, 32), interpolation=Inter.LINEAR)
    rescale_normalization_op = vision.Rescale(1 / 0.3081, -1 * 0.1307 / 0.3081)
    rescale_op = vision.Rescale(1.0 / 255.0, 0.0)
    hwc_to_chw_op = vision.HWC2CHW()
    type_cast_op = transforms.TypeCast(mstype.int32)

    mnist_dataset = mnist_dataset.map(input_columns="label", operations=type_cast_op)
    mnist_dataset = mnist_dataset.map(input_columns="image",
                                      operations=[resize_operation, rescale_op, rescale_normalization_op,
                                                  hwc_to_chw_op])
    mnist_dataset = mnist_dataset.shuffle(buffer_size=10000)
    mnist_dataset = mnist_dataset.batch(batch_size, drop_remainder=True)
    mnist_dataset = mnist_dataset.repeat(repeat_size)

    return mnist_dataset

In [11]:
class LeNet5(nn.Cell):
    """
    LeNet-5 神经网络结构。

    这是一个经典的卷积神经网络，通常用于图像识别任务。它包含了两个卷积层和三个全连接层。

    参数:
        num_class (int): 输出层的类别数量。默认为10，适用于MNIST数据集。
        num_channel (int): 输入图像的通道数。对于灰度图像，此值为1。

    组件:
        - conv1: 第一个卷积层，使用有效填充。
        - conv2: 第二个卷积层，同样使用有效填充。
        - fc1: 第一个全连接层。
        - fc2: 第二个全连接层。
        - fc3: 第三个全连接层，输出层。
        - relu: 激活函数，使用ReLU。
        - max_pool2d: 最大池化层。
        - flatten: 扁平化层，用于全连接层之前的数据转换。

    方法:
        - construct(x): 定义了前向传播的过程。
    """

    def __init__(self, num_class=10, num_channel=1):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(num_channel, 6, 5, pad_mode='valid')
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode='valid')
        self.fc1 = nn.Dense(16 * 5 * 5, 120, weight_init=Normal(0.02))
        self.fc2 = nn.Dense(120, 84, weight_init=Normal(0.02))
        self.fc3 = nn.Dense(84, num_class, weight_init=Normal(0.02))
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def construct(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x



In [27]:
def train_network(model, epoch_size, data_path, repeat_size, checkpoint_callback):
    """
    训练神经网络模型。

    此函数负责初始化数据集，然后使用指定的模型进行训练。在训练过程中，它将记录损失并保存模型的检查点。

    参数:
        model (Model): 要训练的神经网络模型。
        epoch_size (int): 训练过程中遍历数据集的次数。
        data_path (str): 训练数据集的路径。
        repeat_size (int): 数据集的重复次数，用于扩充数据集。
        checkpoint_callback (Callback): 用于保存模型检查点的回调函数。

    过程:
        - 使用 `create_dataset` 函数创建训练数据集。
        - 调用模型的 `train` 方法进行训练。
        - 在训练过程中，会通过回调函数记录损失和保存检查点。

    注意:
        - 确保提供的 `data_path` 包含适当格式的数据。
    """
    print("============== 开始训练 ==============")
    ds_train = create_dataset(data_path, True,32, repeat_size)
    ds_eval = create_dataset(data_path, False, 32, repeat_size)
    model.train(epoch_size, ds_train, callbacks=[checkpoint_callback, LossMonitor(per_print_times=ds_train.get_dataset_size())],
                dataset_sink_mode=False)
    metrics_result = model.eval(ds_eval)
    print('Accuracy:', metrics_result["Accuracy"])
    print("============== 训练结束 ==============")

In [28]:
epochs = 10
data_url = "./data/MNIST_DATA"
output_path = "./check"
net = LeNet5()
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")
net_opt = nn.Momentum(net.trainable_params(), 0.01, 0.9)

model = Model(net, net_loss, net_opt, metrics={"Accuracy": nn.Accuracy()})

config_checkpoint = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=10)
checkpoint_callback = ModelCheckpoint(prefix="checkpoint_lenet", directory=output_path,
                                      config=config_checkpoint)

train_network(model, epochs, data_url, 1, checkpoint_callback)


============== 开始训练 ==============
epoch: 1 step: 1875, loss is 0.017459269613027573
epoch: 2 step: 1875, loss is 0.09903346002101898
epoch: 3 step: 1875, loss is 0.00017907457367982715
epoch: 4 step: 1875, loss is 0.008635335601866245
epoch: 5 step: 1875, loss is 0.19991040229797363
epoch: 6 step: 1875, loss is 0.004170055966824293
epoch: 7 step: 1875, loss is 0.09278905391693115
epoch: 8 step: 1875, loss is 0.0019566493574529886
epoch: 9 step: 1875, loss is 0.0005295849987305701
epoch: 10 step: 1875, loss is 0.0018454601522535086
Accuracy: 0.9878806089743589
============== 训练结束 ==============
